# NER on codiesp-lead using HuggingFace

In [1]:
!pip install transformers
!pip install transformers[torch]
!pip install datasets
!pip install seqeval
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=d8ad023924362bdb63375542a64aa6dd92fc4848aea7c18fad0dcd510eb761f1
  St

In [2]:
!rm -rf *finetuned*
!rm -rf *tmp*
!rm -rf codiesp-main/



In [3]:
!wget https://zenodo.org/record/3837305/files/codiesp.zip
!unzip codiesp.zip -d codiesp

Streaming output truncated to the last 5000 lines.
  inflating: codiesp/final_dataset_v4_to_publish/background/text_files/S0376-78922013000200009-1.txt  
  inflating: codiesp/final_dataset_v4_to_publish/background/text_files/S0376-78922013000400013-1.txt  
  inflating: codiesp/final_dataset_v4_to_publish/background/text_files/S0376-78922014000100011-1.txt  
  inflating: codiesp/final_dataset_v4_to_publish/background/text_files/S0376-78922015000100010-1.txt  
  inflating: codiesp/final_dataset_v4_to_publish/background/text_files/S0376-78922015000100012-1.txt  
  inflating: codiesp/final_dataset_v4_to_publish/background/text_files/S0376-78922015000200006-2.txt  
  inflating: codiesp/final_dataset_v4_to_publish/background/text_files/S0376-78922015000200006-3.txt  
  inflating: codiesp/final_dataset_v4_to_publish/background/text_files/S0376-78922015000200006-4.txt  
  inflating: codiesp/final_dataset_v4_to_publish/background/text_files/S0376-78922015000300010-1.txt  
  inflating: codiesp/f

In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-es"
model_checkpoint = "PlanTL-GOB-ES/bsc-bio-ehr-es"
# model_checkpoint = "PlanTL-GOB-ES/roberta-base-bne"
model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"
# model_checkpoint = "xlm-roberta-large"


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
from transformers import pipeline
unmasker = pipeline('fill-mask', model=model_checkpoint)
unmasker("El único antecedente personal a reseñar era la <mask> arterial.")

[{'score': 0.9731802940368652,
  'token': 3653,
  'token_str': ' hipertensión',
  'sequence': ' El único antecedente personal a reseñar era la hipertensión arterial.'},
 {'score': 0.008870807476341724,
  'token': 11298,
  'token_str': ' Hipertensión',
  'sequence': ' El único antecedente personal a reseñar era la Hipertensión arterial.'},
 {'score': 0.004884412512183189,
  'token': 10306,
  'token_str': ' hipotensión',
  'sequence': ' El único antecedente personal a reseñar era la hipotensión arterial.'},
 {'score': 0.001175886718556285,
  'token': 8694,
  'token_str': ' trombosis',
  'sequence': ' El único antecedente personal a reseñar era la trombosis arterial.'},
 {'score': 0.001038521178998053,
  'token': 5282,
  'token_str': ' HTA',
  'sequence': ' El único antecedente personal a reseñar era la HTA arterial.'}]

In [5]:
import pandas as pd

path_codiesp = 'codiesp'
header_X = ["FILE","TYPE", "CODE", "SUBSTRING", "OFFSETS"]

def get_df_x(split):
  path_x = f"{path_codiesp}/final_dataset_v4_to_publish/{split}/{split}X.tsv"
  return pd.read_csv(path_x, delimiter="\t", names=header_X)

df_x_train = get_df_x('train')

df_x_train.head(25)

,FILE,TYPE,CODE,SUBSTRING,OFFSETS
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163 2171
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787 2801;2810 2823
2,S0004-06142005000700014-1,DIAGNOSTICO,n44.8,teste derecho aumentado de tamaño,1343 1376
3,S0004-06142005000700014-1,DIAGNOSTICO,z20.818,exposición a Brucella,594 615
4,S0004-06142005000700014-1,DIAGNOSTICO,r60.9,edemas,1250 1256
5,S0004-06142005000700014-1,DIAGNOSTICO,r52,dolores,78 85
6,S0004-06142005000700014-1,DIAGNOSTICO,a23.9,Brucella,607 615
7,S0004-06142005000700014-1,DIAGNOSTICO,a23.9,Brucella,2009 2017
8,S0004-06142005000700014-1,DIAGNOSTICO,a23.9,Brucella,2568 2576
9,S0004-06142005000700014-1,DIAGNOSTICO,i83.90,varices,1239 1246


In [6]:
def get_text(split, id):
    fp = open(f'{path_codiesp}/final_dataset_v4_to_publish/{split}/text_files/{id}.txt', 'r')
    txt = fp.read()
    fp.close()
    return txt


print(get_text('train', 'S0004-06142005000900013-1'))

Se trata de una mujer de 29 años sometida a un estudio ecográfico pélvico de control tras una ligadura de trompas por vía laparoscópica. Durante el mismo se detectó una tumoración de 20 mm en la cara lateral derecha de la vejiga, bien delimitada e hipoecoica. La paciente no presentaba sintomatología miccional, como refirió en el interrogatorio posterior.

Se le realizó una urografía intravenosa, en la cual no se detectó ninguna alteración del aparato urinario superior. En el cistograma de la misma se puso de manifiesto un defecto de repleción redondeado y de superficie lisa, localizado en la pared vesical derecha. Las analíticas de sangre y orina estaban dentro de los límites normales. Se le realizó una cistoscopia a la paciente, donde se objetivó la presencia de una tumoración a modo de "joroba", de superficie lisa y mucosa conservada, en cara lateral derecha de vejiga, inmediatamente por encima y delante del meato ureteral ipsilateral.

Con el diagnóstico de presunción de leiomioma v

In [7]:
files = df_x_train["FILE"].unique()
print(len(files))

500


In [8]:
df_raw_x = pd.DataFrame({'file':files})
print(df_raw_x.head(20))

                         file
0   S0004-06142005000700014-1
1   S0004-06142005000900013-1
2   S0004-06142005000900015-1
3   S0004-06142005001000011-3
4   S0004-06142005001000016-1
5   S0004-06142006000100014-1
6   S0004-06142006000200001-1
7   S0004-06142006000200013-1
8   S0004-06142006000200014-1
9   S0004-06142006000300015-1
10  S0004-06142006000500002-2
11  S0004-06142006000500011-1
12  S0004-06142006000600009-1
13  S0004-06142006000600012-1
14  S0004-06142006000700013-1
15  S0004-06142006000700014-1
16  S0004-06142006000900006-1
17  S0004-06142006000900013-1
18  S0004-06142006000900014-1
19  S0004-06142006000900015-1


In [9]:
df_raw_x["text"] = df_raw_x.apply(lambda x: get_text('train', x['file']), axis=1)
df_raw_x.head(2)

,file,text
0,S0004-06142005000700014-1,Describimos el caso de un varón de 37 años con...
1,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...


In [10]:
def get_tokens(text):
        return tokenizer(text, truncation=False)

df_raw_x["tokens"] = df_raw_x['text'].apply(get_tokens)
df_raw_x.head(2)

Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors


,file,text,tokens
0,S0004-06142005000700014-1,Describimos el caso de un varón de 37 años con...,"[input_ids, attention_mask]"
1,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...,"[input_ids, attention_mask]"


In [11]:
def get_token_length(tokens):
  return len(tokens['input_ids'])

df_raw_x["token_len"] = df_raw_x['tokens'].apply(get_token_length)
df_raw_x["token_len"].describe()

count     500.000000
mean      458.126000
std       223.513478
min       109.000000
25%       293.000000
50%       409.000000
75%       568.500000
max      1488.000000
Name: token_len, dtype: float64

In [12]:
import re

raw_text = df_raw_x["text"].to_list()
max_length = 256
sections = []
sections_len = []


def get_sections_max_length(section):
  section_list = []
  sentences = re.split("(\\.\s)", section)
  sub_section = ""
  sub_section_tokens_len = get_token_length(get_tokens(sub_section))
  for sent in sentences:
    sent_token_length = get_token_length(get_tokens(sent))

    if sub_section_tokens_len + sent_token_length < max_length:
      sub_section = sub_section + sent
      sub_section_tokens_len = sub_section_tokens_len + sent_token_length
    else:
      section_list.append(sub_section)
      sub_section = sent
      sub_section_tokens_len = sent_token_length

  if sub_section:
    section_list.append(sub_section)

  return section_list


def get_sections(text):
  sections_len = []
  sections = text.split('\n\n')

  for section in sections:
    if not section:
      continue

    if get_token_length(get_tokens(section)) < max_length:
      sections_len.append(section)
    else:
      sections_len.extend(get_sections_max_length(section))

  return sections_len


df_raw_x["sections"] = df_raw_x["text"].apply(get_sections)


In [14]:
def get_offsets(text, list_substrings):
  offsets = []
  for section in list_substrings:
    offsets.append(text.find(section))
  return offsets

file = df_raw_x['file'].iloc[0]
text = df_raw_x['text'].iloc[0]
sections = df_raw_x['sections'].iloc[0]

print(get_offsets(text, sections))
print(len(sections))
for section in sections:
  print('FILE', file)
  print('SECTION:')
  print(section)
  print('\n')


[0, 1023, 1738, 2515]
4
FILE S0004-06142005000700014-1
SECTION:
Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias. Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado. Entre los comensales aparecieron varios casos de brucelosis. Durante el ingreso para estudio del síndrome febril con antecedentes epidemiológicos de posible exposición a Brucella presenta un cuadro de orquiepididimitis derecha.
La exploración física revela: Tª 40,2 C; T.A: 109/68 mmHg; Fc: 105 lpm. Se encuentra consciente, orientado, sudoroso, eupneico, con buen estado de nutrición e hidratación. En cabeza y cuello no se palpan adenopatías, ni bocio ni ingurgitación de vena yugular, con pulsos 

In [15]:
df_raw_x['section_offsets'] = df_raw_x.apply(lambda x: get_offsets(x['text'], x['sections']), axis=1)
df_raw_x.head(2)

,file,text,tokens,token_len,sections,section_offsets
0,S0004-06142005000700014-1,Describimos el caso de un varón de 37 años con...,"[input_ids, attention_mask]",657,[Describimos el caso de un varón de 37 años co...,"[0, 1023, 1738, 2515]"
1,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...,"[input_ids, attention_mask]",359,[Se trata de una mujer de 29 años sometida a u...,"[0, 358, 953, 1554]"


In [16]:
df_x_train_sel = df_x_train[df_x_train["FILE"] == "S0004-06142005000900013-1"]
df_x_train_sel

,FILE,TYPE,CODE,SUBSTRING,OFFSETS
20,S0004-06142005000900013-1,DIAGNOSTICO,d30.3,leiomioma vesical,989 1006
21,S0004-06142005000900013-1,PROCEDIMIENTO,0tjb8zz,cistoscopia,1611 1622
22,S0004-06142005000900013-1,PROCEDIMIENTO,0ttb,resección vejiga,1773 1782;222 228
23,S0004-06142005000900013-1,PROCEDIMIENTO,0ul7,ligadura de trompas,94 113
24,S0004-06142005000900013-1,DIAGNOSTICO,r58,sangrado,1188 1196
25,S0004-06142005000900013-1,PROCEDIMIENTO,bw4gzzz,estudio ecográfico pélvico,47 73
26,S0004-06142005000900013-1,PROCEDIMIENTO,0tjb8zz,cistoscopia,713 724


In [17]:
df_raw_x_sel = df_raw_x[df_raw_x["file"] == "S0004-06142005000900013-1"]
df_raw_x_sel

,file,text,tokens,token_len,sections,section_offsets
1,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...,"[input_ids, attention_mask]",359,[Se trata de una mujer de 29 años sometida a u...,"[0, 358, 953, 1554]"


In [18]:
def get_entity_tuples(tuple_string):

  entities_tuples = []
  if ";" in tuple_string:
    entity_offsets_list = tuple_string.split(";")
  else:
    entity_offsets_list = [tuple_string]

  for item in entity_offsets_list:
    item_s = item.split(" ")
    item_tuple = (int(item_s[0]), int(item_s[1]))
    entities_tuples.append(item_tuple)
  return entities_tuples

df_x_train_sel["OFFSETS_T"] = df_x_train_sel["OFFSETS"].apply(get_entity_tuples)
df_x_train_sel.head(20)

<ipython-input-18-bc8ddd2d8314>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_train_sel["OFFSETS_T"] = df_x_train_sel["OFFSETS"].apply(get_entity_tuples)


,FILE,TYPE,CODE,SUBSTRING,OFFSETS,OFFSETS_T
20,S0004-06142005000900013-1,DIAGNOSTICO,d30.3,leiomioma vesical,989 1006,"[(989, 1006)]"
21,S0004-06142005000900013-1,PROCEDIMIENTO,0tjb8zz,cistoscopia,1611 1622,"[(1611, 1622)]"
22,S0004-06142005000900013-1,PROCEDIMIENTO,0ttb,resección vejiga,1773 1782;222 228,"[(1773, 1782), (222, 228)]"
23,S0004-06142005000900013-1,PROCEDIMIENTO,0ul7,ligadura de trompas,94 113,"[(94, 113)]"
24,S0004-06142005000900013-1,DIAGNOSTICO,r58,sangrado,1188 1196,"[(1188, 1196)]"
25,S0004-06142005000900013-1,PROCEDIMIENTO,bw4gzzz,estudio ecográfico pélvico,47 73,"[(47, 73)]"
26,S0004-06142005000900013-1,PROCEDIMIENTO,0tjb8zz,cistoscopia,713 724,"[(713, 724)]"


In [19]:
def get_tag(type, begin):
  if type == "DIAGNOSTICO":
    if begin:
      return 1
    else:
      return 2
  elif type == "PROCEDIMIENTO":
    if begin:
      return 3
    else:
      return 4
  else:
    return 0


def get_token_tags(offset_section, tokens, df_x_train_sel):
  tags = [0 for token in tokens['input_ids']]
  for idx, row in df_x_train_sel.iterrows():
    tuples = get_entity_tuples(row["OFFSETS"])
    tuples_currected = [(item[0] - offset_section, item[1] - offset_section) for item in tuples]

    for entity_span in tuples_currected:
      entity = False
      for idx_offset_mapping, offset_mapping in enumerate(tokens['offset_mapping']):
        if offset_mapping[0] == entity_span[0]:
          entity = True
          # print('match', row["CODE"], row["SUBSTRING"], offset_mapping, entity_span, tokenizer.decode(tokens['input_ids'][idx_offset_mapping]))
          tags[idx_offset_mapping] = get_tag(row["TYPE"], True)
        elif entity:
          if offset_mapping[1] <= entity_span[1]:
            tags[idx_offset_mapping] = get_tag(row["TYPE"], False)
          else:
            entity = False
  return tags


def tokenize_offsets(txt):
  return tokenizer(txt, is_split_into_words=False, return_offsets_mapping=True)




In [20]:
def create_json(split, main=False):
  df_x_split = None
  if main:
      df_x_split = get_df_x_main(split)
  else:
    df_x_split = get_df_x(split)

  files = df_x_split["FILE"].unique()

  df_raw_x_split = pd.DataFrame({'file':files})
  df_raw_x_split["text"] = df_raw_x_split.apply(lambda x: get_text(split, x['file']), axis=1)
  df_raw_x_split["sections"] = df_raw_x_split["text"].apply(get_sections)
  df_raw_x_split['section_offsets'] = df_raw_x_split.apply(lambda x: get_offsets(x['text'], x['sections']), axis=1)

  print(df_raw_x_split.head())

  sections = []
  for idx_raw, row in df_raw_x_split.iterrows():

    file_name = row["file"]
    df_x_split_sel = df_x_split[df_x_split["FILE"] == file_name]

    for idx, item in enumerate(row["sections"]):
      section = {}

      section["TEXT"] = row["sections"][idx]
      section["FILE"] = file_name
      section["NR"] = idx
      section["OFFSET"] = row["section_offsets"][idx]


      tokens = tokenize_offsets(row["sections"][idx])
      section["IDS"] = tokens['input_ids']
      section["OFFSETS"] = tokens['offset_mapping']
      section["TOKENS"] = [tokenizer.decode(token_id) for token_id in tokens['input_ids']]

      section["TAGS"] = get_token_tags(section["OFFSET"], tokens, df_x_split_sel)
      sections.append(section)
      if section["TAGS"]:
        merged = [(section["TOKENS"][i], section["TAGS"][i]) for i in range(0, len(section["TAGS"]))]
      # print(section["TEXT"] )
      # print(merged)
  return sections

train_sections = create_json('train')


                        file  \
0  S0004-06142005000700014-1   
1  S0004-06142005000900013-1   
2  S0004-06142005000900015-1   
3  S0004-06142005001000011-3   
4  S0004-06142005001000016-1   

                                                text  \
0  Describimos el caso de un varón de 37 años con...   
1  Se trata de una mujer de 29 años sometida a un...   
2  Varón de 36 años, sin antecedentes de interés,...   
3  Mujer de 42 años en el momento de someterse a ...   
4  Varón de 65 años que consultaba por un síndrom...   

                                            sections        section_offsets  
0  [Describimos el caso de un varón de 37 años co...  [0, 1023, 1738, 2515]  
1  [Se trata de una mujer de 29 años sometida a u...    [0, 358, 953, 1554]  
2  [Varón de 36 años, sin antecedentes de interés...              [0, 1020]  
3  [Mujer de 42 años en el momento de someterse a...        [0, 1064, 2191]  
4  [Varón de 65 años que consultaba por un síndro...          [0, 653, 769]  


In [21]:
def write_json(sections, split):
  sections_fields = [{"text": item["TEXT"], "tokens": item["TOKENS"], "ner_tags": item["TAGS"], "file": item["FILE"], "offset": item["OFFSET"],  "nr": item["NR"] } for item in sections]
  df = pd.DataFrame.from_records(sections_fields)
  df.to_json(f"codiesp-ner-{split}.jsonl", orient="records", lines=True, force_ascii=False)

write_json(train_sections, 'train')

In [22]:
dev_sections = create_json('dev')
write_json(dev_sections, 'dev')

test_sections = create_json('test')
write_json(test_sections, 'test')

                        file  \
0  S0004-06142005000900016-1   
1  S0004-06142005001000011-1   
2  S0004-06142006000200011-1   
3  S0004-06142006000500002-3   
4  S0004-06142006000500002-4   

                                                text  \
0  Mujer de 29 años con antecedentes de ulcus duo...   
1  Varón de 58 años de edad en el momento del tra...   
2  Paciente varón de 22 años de edad, sin anteced...   
3  Paciente de 35 años que nos fue remitido al se...   
4  Paciente de 90 años que acude a su Urólogo de ...   

                                            sections       section_offsets  
0  [Mujer de 29 años con antecedentes de ulcus du...        [0, 850, 1251]  
1  [Varón de 58 años de edad en el momento del tr...       [0, 1104, 2142]  
2  [Paciente varón de 22 años de edad, sin antece...  [0, 940, 2152, 2260]  
3  [Paciente de 35 años que nos fue remitido al s...       [0, 1088, 1982]  
4  [Paciente de 90 años que acude a su Urólogo de...       [0, 1071, 2012]  
        

In [23]:
!unzip codiesp-main.zip -d codiesp-main

Archive:  codiesp-main.zip
  inflating: codiesp-main/train_main_x.xlsx  
  inflating: codiesp-main/__MACOSX/._train_main_x.xlsx  
  inflating: codiesp-main/dev_main_x.xlsx  
  inflating: codiesp-main/__MACOSX/._dev_main_x.xlsx  
  inflating: codiesp-main/test_main_x.xlsx  
  inflating: codiesp-main/__MACOSX/._test_main_x.xlsx  


In [24]:
path_codiesp_main = 'codiesp-main'

def get_df_x_main(split):
  path_x = f"{path_codiesp_main}/{split}_main_x.xlsx"
  df_x = pd.read_excel(path_x)

  # quick manual bugfixing for some missing mappings
  substrings = df_x["SUBSTRING"].to_list()
  mains = df_x["MAIN"].to_list()
  offsets_main = df_x["MAIN_OFFSETS"].to_list()
  offsets = df_x["OFFSETS"].to_list()
  for idx, offset in enumerate(offsets_main):
    if type(offset) is not str:
      print(idx, offset, substrings[idx])
      main = substrings[idx].split(" ")[0]
      start = str(int(offsets[idx].split(" ")[0]))
      end = str(int(start) + len(main))
      offset_corrected = " ".join([start, end])
      mains[idx] = main
      offsets_main[idx] = offset_corrected
      print(idx, offset_corrected, main)
  df_x["MAIN"] = mains
  df_x["MAIN_OFFSETS"] = offsets_main


  df_x["SUBSTRING"] = df_x["MAIN"]
  df_x["OFFSETS"] = df_x["MAIN_OFFSETS"]
  df_x = df_x[df_x.columns.intersection(header_X)]
  return df_x

get_df_x_main('train').head()
get_df_x_main('test').head()
get_df_x_main('dev').head()



253 nan dilatación de la uretra
253 1492 1502 dilatación
3887 nan dilatación de asas yeyunales
3887 761 771 dilatación
2391 nan dilatación de colédoco
2391 1205 1215 dilatación
2408 nan dilatación de vía biliar
2408 515 525 dilatación
125 nan diálisis peritoneal
125 260 268 diálisis
1279 nan taponamiento nasal izquierdo
1279 468 480 taponamiento


,FILE,TYPE,CODE,SUBSTRING,OFFSETS
0,S0004-06142005000900016-1,DIAGNOSTICO,q62.11,estenosis,540 549
1,S0004-06142005000900016-1,DIAGNOSTICO,n28.89,ectasia,326 333
2,S0004-06142005000900016-1,DIAGNOSTICO,n39.0,infecciones,198 209
3,S0004-06142005000900016-1,DIAGNOSTICO,r31.9,hematuria,185 194
4,S0004-06142005000900016-1,DIAGNOSTICO,n23,dolor,85 90


In [25]:
train_sections = create_json('train', main=True)
write_json(train_sections, 'train')
print('written train')
dev_sections = create_json('dev', main=True)
write_json(dev_sections, 'dev')
print('written dev')
test_sections = create_json('test', main=True)
write_json(test_sections, 'test')
print('written test')


253 nan dilatación de la uretra
253 1492 1502 dilatación
3887 nan dilatación de asas yeyunales
3887 761 771 dilatación
                        file  \
0  S0004-06142005000700014-1   
1  S0004-06142005000900013-1   
2  S0004-06142005000900015-1   
3  S0004-06142005001000011-3   
4  S0004-06142005001000016-1   

                                                text  \
0  Describimos el caso de un varón de 37 años con...   
1  Se trata de una mujer de 29 años sometida a un...   
2  Varón de 36 años, sin antecedentes de interés,...   
3  Mujer de 42 años en el momento de someterse a ...   
4  Varón de 65 años que consultaba por un síndrom...   

                                            sections        section_offsets  
0  [Describimos el caso de un varón de 37 años co...  [0, 1023, 1738, 2515]  
1  [Se trata de una mujer de 29 años sometida a u...    [0, 358, 953, 1554]  
2  [Varón de 36 años, sin antecedentes de interés...              [0, 1020]  
3  [Mujer de 42 años en el momento de s

In [26]:
token_length = [get_token_length(get_tokens(item["TEXT"])) for item in dev_sections]
print(max(token_length))
token_length = [get_token_length(get_tokens(item["TEXT"])) for item in train_sections]
print(max(token_length))
token_length = [get_token_length(get_tokens(item["TEXT"])) for item in test_sections]
print(max(token_length))



255
419
262


In [27]:
from datasets import Features, Sequence, Value
from datasets.features import ClassLabel

_DESCRIPTION = "CodiEsp NER"

features = Features(
        {
            "text": Value("string"),
            "tokens": Sequence(Value("string")),
            "file": Value("string"),
            "nr": Value("int8"),
            "offset": Value("int32"),
            "ner_tags": Sequence(ClassLabel(names=["O","B-D", "I-D", "B-P", "I-P"]))
        }
)


In [28]:
from datasets import load_dataset
ds = load_dataset("json", data_files={
    "train":"codiesp-ner-train.jsonl",
    "validation":"codiesp-ner-dev.jsonl",
    "test":"codiesp-ner-test.jsonl"},
    features=features)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-087f9f71f8691553/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [29]:
ds['train'][40]

{'text': 'Varón de 80 años que debutó clínicamente con hematuria y dolor en flanco izquierdo. La puñopercusión en fosa renal izquierda fue positiva. Mediante técnicas de imagen se observó una masa a nivel de fosa renal izquierda que desdibujaba la silueta renal. El paciente fue intervenido realizándose una nefrectomía con ureterectomía proximal.\nEn la pieza de nefrectomía el parénquima renal estaba sustituido en su práctica totalidad por una lesión blanquecina heterogénea en cuyo seno se podían apreciar cavidades quísticas de aspecto necrótico con formaciones papilares friables y mal delimitadas intracavitarias y áreas amarillentas mal delimitadas combinadas al azar con otras de aspecto necrótico y/o hemorrágico. Al corte se observó una consistencia firme, fundamentalmente correspondiendo con las áreas blanquecinogrisaceas si bien en otras localizaciones se apreció una consistencia blanda e incluso crujiente.\nEl estudio histológico demostró una neoplasia constituida por una prolifera

In [30]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    for idx, word_id in enumerate(word_ids):
        if word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Labels are already at token index
            new_labels.append(labels[idx])

    return new_labels

In [31]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["text"], truncation=False, is_split_into_words=False
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [32]:
inputs = tokenizer(ds["train"][0]["text"], is_split_into_words=False)
inputs.tokens()

['<s>',
 'ĠDescribimos',
 'Ġel',
 'Ġcaso',
 'Ġde',
 'Ġun',
 'ĠvarÃ³n',
 'Ġde',
 'Ġ37',
 'ĠaÃ±os',
 'Ġcon',
 'Ġvida',
 'Ġprevia',
 'Ġactiva',
 'Ġque',
 'Ġrefiere',
 'Ġdolores',
 'Ġosteo',
 'articulares',
 'Ġde',
 'ĠlocalizaciÃ³n',
 'Ġvariable',
 'Ġen',
 'Ġel',
 'ĠÃºltimo',
 'Ġmes',
 'Ġy',
 'Ġfiebre',
 'Ġen',
 'Ġla',
 'ĠÃºltima',
 'Ġsemana',
 'Ġcon',
 'Ġpicos',
 'Ġ(',
 'ma',
 'tu',
 'tino',
 'Ġy',
 'Ġvesper',
 'tino',
 ')',
 'Ġde',
 'Ġ40',
 'ĠC',
 'Ġlas',
 'ĠÃºltimas',
 'Ġ24',
 '-',
 '48',
 'Ġhoras',
 ',',
 'Ġpor',
 'Ġlo',
 'Ġque',
 'Ġacude',
 'Ġal',
 'ĠServicio',
 'Ġde',
 'ĠUrgencias',
 '.',
 'ĠAntes',
 'Ġde',
 'Ġcomenzar',
 'Ġel',
 'Ġcuadro',
 'Ġestuvo',
 'Ġen',
 'ĠExtremadura',
 'Ġen',
 'Ġuna',
 'ĠregiÃ³n',
 'ĠendÃ©mica',
 'Ġde',
 'Ġbruc',
 'ella',
 ',',
 'Ġingi',
 'riendo',
 'Ġleche',
 'Ġde',
 'Ġcabra',
 'Ġsin',
 'Ġpasteuri',
 'zar',
 'Ġy',
 'Ġqueso',
 'Ġde',
 'Ġdicho',
 'Ġganado',
 '.',
 'ĠEntre',
 'Ġlos',
 'Ġcomens',
 'ales',
 'Ġaparecieron',
 'Ġvarios',
 'Ġcasos',
 'Ġde',
 'Ġbruc'

In [33]:
print(len(inputs.word_ids()))
print(ds["train"][0]["ner_tags"])
print(ds["train"][0]["tokens"])

# print(len(input.))

214
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['<s>', ' Describimos', ' el', ' caso', ' de', ' un', ' varón', ' de', ' 37', ' años', ' con', ' vida', ' previa', ' activa', ' que', ' refiere', ' dolores', ' osteo', 'articulares', ' de', ' localización', ' variable', ' en', ' el', ' último', ' mes', ' y', ' fiebre', ' en', ' la', ' última', ' semana', ' con', ' picos', ' (', 'ma', 'tu', 'tino', ' y

In [34]:
labels = ds["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,

In [35]:
tokenize_and_align_labels(ds['train'][:60])

{'input_ids': [[0, 47522, 297, 990, 262, 308, 8276, 262, 3684, 610, 299, 898, 3664, 4288, 300, 2324, 6744, 4977, 35017, 262, 6399, 5305, 288, 297, 2447, 968, 290, 2389, 288, 284, 3403, 2092, 299, 20477, 350, 372, 348, 2983, 290, 43116, 2983, 12, 262, 2015, 329, 347, 5641, 1672, 16, 5085, 1257, 15, 344, 424, 300, 2839, 337, 2607, 262, 3599, 17, 7405, 262, 6811, 297, 2888, 6047, 288, 13308, 288, 363, 2689, 25997, 262, 45849, 5248, 15, 36341, 5316, 4221, 262, 45304, 526, 43260, 1297, 290, 17112, 262, 2565, 10779, 17, 2695, 318, 42196, 394, 22279, 2237, 1006, 262, 45849, 27050, 17, 2834, 297, 2680, 361, 754, 331, 2264, 9132, 299, 3214, 12226, 262, 1414, 3292, 269, 46075, 5248, 1752, 308, 2888, 262, 574, 488, 12157, 16606, 27739, 2550, 17, 202, 2635, 3904, 2469, 11405, 29, 403, 130, 107, 2015, 15, 21, 329, 30, 403, 17, 36, 29, 14931, 18, 2818, 4126, 30, 23247, 29, 11404, 4298, 17, 551, 2194, 5827, 15, 7665, 15, 14867, 1225, 15, 13569, 15, 299, 1087, 1302, 262, 5603, 277, 10169, 17, 452, 315

In [36]:
tokenized_datasets = ds.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1487 [00:00<?, ? examples/s]

Map:   0%|          | 0/787 [00:00<?, ? examples/s]

Map:   0%|          | 0/782 [00:00<?, ? examples/s]

In [37]:
task = "ner"
label_list = ds["train"].features[f"{task}_tags"].feature.names


In [38]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
import evaluate

metric = evaluate.load("seqeval")

In [40]:
ner_feature = ds["train"].features["ner_tags"]

label_names = ner_feature.feature.names

labels = ds["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-D',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-D',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-D',
 'I-D',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-D',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-D',
 'O',
 'B-D',
 'I-D',
 'O',
 'O',
 'O',
 'O',
 'B-D',
 'I-D',
 'I-D',
 'I-D',
 'I-D',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '

In [ ]:
# predictions = labels.copy()
# predictions[2] = "O"
# metric.compute(predictions=[predictions], references=[labels])

In [41]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [42]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

print(id2label)

{0: 'O', 1: 'B-D', 2: 'I-D', 3: 'B-P', 4: 'I-P'}


In [43]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
model.config.num_labels


5

In [45]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 4

args = TrainingArguments(
    f"{model_name}-codiesp-ner-finetuned",
    evaluation_strategy = "epoch",
    save_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=False,
    push_to_hub=False,
)

In [46]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from transformers import Trainer
!rm -rf *finetuned*

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.093019,0.826667,0.783245,0.804370,0.987363
2,0.001500,0.085664,0.782055,0.837434,0.808798,0.986299
3,0.001600,0.089230,0.769786,0.850399,0.808087,0.986238
4,0.001600,0.093363,0.766766,0.851396,0.806868,0.986036
5,0.001500,0.086337,0.803099,0.827128,0.814936,0.987275
6,0.001400,0.082975,0.796740,0.828790,0.812449,0.986889
7,0.001000,0.087379,0.810345,0.828125,0.819138,0.987486
8,0.001000,0.088783,0.800128,0.833112,0.816287,0.987100
9,0.000500,0.092254,0.794848,0.841090,0.817315,0.987073
10,0.000400,0.091829,0.810513,0.830452,0.820361,0.987319


TrainOutput(global_step=3720, training_loss=0.0010849802224065668, metrics={'train_runtime': 867.446, 'train_samples_per_second': 17.142, 'train_steps_per_second': 4.288, 'total_flos': 1618308292603470.0, 'train_loss': 0.0010849802224065668, 'epoch': 10.0})

In [ ]:
# !rm -rf *finetuned*
trainer.save_model()

In [ ]:
model_name_ner = f"{model_name}-codiesp-ner-finetuned"
!zip -r {model_name_ner}.zip {model_name_ner}

  adding: roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/ (stored 0%)
  adding: roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/tokenizer_config.json (deflated 81%)
  adding: roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/training_args.bin (deflated 48%)
  adding: roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/vocab.json (deflated 60%)
  adding: roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/pytorch_model.bin (deflated 12%)
  adding: roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/tokenizer.json (deflated 72%)
  adding: roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/runs/ (stored 0%)
  adding: roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/runs/Jul19_14-59-49_4c6cabdbbc32/ (stored 0%)
  adding: roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/runs/Jul19_14-59-49_4c6cabdbbc32/events.out.tfevents.1689778798.4c6cabdbbc32.558.4 (deflated 62%)
  adding: roberta-base-biomedical-clinical-es-codiesp-

In [ ]:
trainer.eval_dataset =tokenized_datasets["test"]
trainer.evaluate()

{'eval_loss': 0.09046100825071335,
 'eval_precision': 0.8103022825416409,
 'eval_recall': 0.833174754202347,
 'eval_f1': 0.8215793588741204,
 'eval_accuracy': 0.9879377806566819,
 'eval_runtime': 16.4457,
 'eval_samples_per_second': 47.55,
 'eval_steps_per_second': 11.918,
 'epoch': 10.0}

In [47]:
!rm -rf *finetuned*
!unzip /content/drive/MyDrive/CodiEsp/roberta-base-biomedical-clinical-es-codiesp-ner-finetuned.zip -d ./

Archive:  /content/drive/MyDrive/CodiEsp/roberta-base-biomedical-clinical-es-codiesp-ner-finetuned.zip
   creating: ./roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/
  inflating: ./roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/tokenizer_config.json  
  inflating: ./roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/training_args.bin  
  inflating: ./roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/vocab.json  
  inflating: ./roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/pytorch_model.bin  
  inflating: ./roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/tokenizer.json  
   creating: ./roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/runs/
   creating: ./roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/runs/Jul19_14-59-49_4c6cabdbbc32/
  inflating: ./roberta-base-biomedical-clinical-es-codiesp-ner-finetuned/runs/Jul19_14-59-49_4c6cabdbbc32/events.out.tfevents.1689778798.4c6cabdbbc32.558.4  
  inflating: ./rober

In [48]:
from transformers import pipeline

token_classifier = pipeline(
    "token-classification",
    model="roberta-base-biomedical-clinical-es-codiesp-ner-finetuned",
    aggregation_strategy="simple",
    device=0
)
token_classifier(tokenized_datasets["validation"][0]['text'])

[{'entity_group': 'D',
  'score': 0.99982023,
  'word': ' ulcus',
  'start': 37,
  'end': 42},
 {'entity_group': 'D',
  'score': 0.99978524,
  'word': ' estreñimiento',
  'start': 54,
  'end': 67},
 {'entity_group': 'D',
  'score': 0.99578935,
  'word': ' dolor',
  'start': 85,
  'end': 90},
 {'entity_group': 'D',
  'score': 0.99980175,
  'word': ' nefrolitiasis',
  'start': 168,
  'end': 181},
 {'entity_group': 'D',
  'score': 0.9997907,
  'word': ' hematuria',
  'start': 185,
  'end': 194},
 {'entity_group': 'D',
  'score': 0.99987733,
  'word': ' infecciones',
  'start': 198,
  'end': 209},
 {'entity_group': 'D',
  'score': 0.9998031,
  'word': ' ectasia',
  'start': 326,
  'end': 333},
 {'entity_group': 'D',
  'score': 0.9998462,
  'word': ' estenosis',
  'start': 540,
  'end': 549}]

In [49]:
tokenized_datasets["validation"][0]['text']

'Mujer de 29 años con antecedentes de ulcus duodenal y estreñimiento que consulta por dolor en fosa renal derecha compatible con crisis renoureteral. No antecedentes de nefrolitiasis ni hematuria ni infecciones del tracto urinario. En la exploración sólo destaca una puñopercusión renal derecha positiva. La ecografía objetiva ectasia pielocalicial renal derecha con adelgazamiento del parénquima. La UIV muestra una anulación funcional de la unidad renal derecha siendo normal el resto de la exploración. Una pielografía retrograda muestra estenosis en la unión pieloureteral derecha, siendo la citología urinaria selectiva del uréter derecho negativa. Ante la disyuntiva de practicar una cirugía reconstructiva o una exerética se realiza gammagrafía renal que demuestra captación relativa del 33% para el riñón derecho y del 67% para el izquierdo.'

In [59]:
from tqdm import tqdm
import json

def map_item(item):
  item['score'] = float(item['score'])
  return item

def get_entities(split):
  entities = []
  for idx, item in enumerate(tqdm(tokenized_datasets[split])):
    entity = {}
    entity["file"] = item["file"]
    entity["nr"] = item["nr"]
    entity["text"] = item ["text"]
    entity["offset"] = item["offset"]
    item_entities = token_classifier(item['text'])
    item_entities = [map_item(item) for item in item_entities]
    entity["entities"] = json.dumps(item_entities, ensure_ascii=False)
    entities.append(entity)
  return entities

In [63]:
for split in ["train", "validation", "test"]:
  print(split)
  entities = get_entities(split)
  df_entities = pd.DataFrame().from_records(entities)
  df_entities.to_excel(f"entities-ner-{split}.xlsx")

train


  0%|          | 0/1487 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 1487/1487 [00:26<00:00, 56.67it/s]


validation


  0%|          | 0/787 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 787/787 [00:13<00:00, 58.91it/s]


test


  0%|          | 0/782 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 782/782 [00:13<00:00, 56.72it/s]


In [61]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "roberta-base-biomedical-clinical-es-codiesp-ner-finetuned",
    id2label=id2label,
    label2id=label2id,
)

batch_size = 4

args = TrainingArguments(
    "roberta-base-biomedical-clinical-es-codiesp-ner-finetuned",
    evaluation_strategy = "epoch",
    save_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.eval_dataset =tokenized_datasets["test"]
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.09046100825071335,
 'eval_precision': 0.8103022825416409,
 'eval_recall': 0.833174754202347,
 'eval_f1': 0.8215793588741204,
 'eval_accuracy': 0.9879377806566819,
 'eval_runtime': 13.2904,
 'eval_samples_per_second': 58.839,
 'eval_steps_per_second': 14.747}

In [64]:
!tar -cvf codiesp-main-predict.tar entities-ner-train.xlsx entities-ner-validation.xlsx entities-ner-test.xlsx

entities-ner-train.xlsx
entities-ner-validation.xlsx
entities-ner-test.xlsx


In [65]:
import torch
torch.cuda.mem_get_info()


(13837860864, 15835398144)